<a href="https://colab.research.google.com/github/Nomad653/Python/blob/main/Titanic_competition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [108]:
import numpy as np

In [109]:
import pandas as pd

In [110]:
from sklearn.linear_model import LogisticRegression

In [111]:
from sklearn.preprocessing import StandardScaler

In [112]:
from sklearn.pipeline import Pipeline

In [113]:
from sklearn.impute import SimpleImputer

In [114]:
from sklearn.preprocessing import OneHotEncoder

In [115]:
from sklearn.compose import ColumnTransformer

In [116]:
from sklearn.ensemble import RandomForestClassifier

In [117]:
from sklearn.model_selection import train_test_split

In [118]:
from scipy.stats import zscore

In [119]:
from sklearn import tree

In [120]:
from sklearn.decomposition import PCA

In [121]:
from sklearn.model_selection import GridSearchCV

In [122]:
data = pd.read_csv("/content/drive/MyDrive/Machine Learning/Titanic competition/train.csv")

In [123]:
new_data = data.drop(columns=["Name"])

In [124]:
new_data["Group"] = new_data['PassengerId'].apply(lambda x: x.split('_')[0]).astype(int)

In [125]:
new_data["Group_size"] = new_data['Group'].map(lambda x: new_data['Group'].value_counts()[x])

In [126]:
new_data['IsAlone'] = np.where((new_data["Group_size"] == 1),1,0)

In [127]:
new_data.drop(columns=["PassengerId"],inplace=True)

In [128]:
new_data['Total_billed'] = new_data.RoomService + new_data.FoodCourt + new_data.ShoppingMall +new_data.Spa +new_data.VRDeck

In [129]:
conditions = [
              (new_data['Age']<2),
              (new_data['Age']<=15), #Child
              (new_data['Age']>15) & (new_data['Age']<=18), #Teenager
              (new_data['Age']>18) & (new_data['Age']<=35),#Young adult
              (new_data['Age']>35) & (new_data['Age']<=50), #Mid-age
              (new_data['Age']>50) & (new_data['Age']<=60),#Old
              (new_data['Age']>60)


]
values = ['Infant','Child','Teenager','Young adult','Mid-age','Old','Elderly']

In [130]:
new_data['Age_group'] = np.select(conditions,values)

In [131]:
new_data[["cabin_param_1","cabin_param_2","cabin_param_3"]]=new_data['Cabin'].str.split("/",expand=True)

In [132]:
new_data['cabin_param_1']=new_data['cabin_param_1'].map({'B':1, 'F':5, 'A':0, 'G':6 ,'E':4, 'D':3, 'C':2, 'T':7})

In [133]:
new_data['VIP'] = new_data['VIP'].map({True:1,False:0})

In [134]:
new_data['CryoSleep'] = new_data['CryoSleep'].map({True:1,False:0})

In [135]:
new_data['cabin_param_3']=new_data['cabin_param_3'].map({'P':0,  'S':1})

In [136]:
new_data.drop(columns=["Cabin"],inplace=True)

In [137]:
new_data["Age_group"] = new_data["Age_group"].map({"Infant":0,"Child":1,"Teenager":2,"Young adult":3,"Mid-age":4,"Old":5,"Elderly":6})

In [167]:
new_data.corr()

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Group,Group_size,IsAlone,Total_billed,Age_group,cabin_param_1,cabin_param_3
CryoSleep,1.000000,-0.074273,-0.081402,-0.252396,-0.211510,-0.212514,-0.203991,-0.198857,0.468645,-0.005879,0.082651,-0.095697,-0.385749,-0.098673,0.017823,0.019677
Age,-0.074273,1.000000,0.092819,0.068723,0.130421,0.033133,0.123970,0.101007,-0.075026,-0.009439,-0.179102,0.135174,0.189475,0.937267,-0.248003,0.011640
VIP,-0.081402,0.092819,1.000000,0.058785,0.129799,0.018295,0.061059,0.125974,-0.037650,0.014018,0.002758,-0.034373,0.170024,0.080972,-0.183393,-0.010117
RoomService,-0.252396,0.068723,0.058785,1.000000,-0.015889,0.054480,0.010080,-0.019581,-0.244611,-0.000346,-0.040777,0.020116,0.235768,0.071395,-0.026783,-0.012290
FoodCourt,-0.211510,0.130421,0.129799,-0.015889,1.000000,-0.014228,0.221891,0.227995,0.046566,-0.009552,0.032632,-0.067533,0.747214,0.123087,-0.324348,0.020149
ShoppingMall,-0.212514,0.033133,0.018295,0.054480,-0.014228,1.000000,0.013879,-0.007322,0.010141,0.017397,-0.038770,0.029211,0.224083,0.043715,-0.032926,-0.020983
Spa,-0.203991,0.123970,0.061059,0.010080,0.221891,0.013879,1.000000,0.153821,-0.221131,-0.005211,0.019223,-0.043821,0.597796,0.118069,-0.229111,0.002031
VRDeck,-0.198857,0.101007,0.125974,-0.019581,0.227995,-0.007322,0.153821,1.000000,-0.207075,0.015432,0.009401,-0.044424,0.573186,0.100275,-0.257326,-0.008154
Transported,0.468645,-0.075026,-0.037650,-0.244611,0.046566,0.010141,-0.221131,-0.207075,1.000000,0.021491,0.082644,-0.113792,-0.197671,-0.098055,-0.116462,0.103775
Group,-0.005879,-0.009439,0.014018,-0.000346,-0.009552,0.017397,-0.005211,0.015432,0.021491,1.000000,0.014753,-0.000266,-0.001793,-0.005843,-0.002276,-0.006721


In [139]:
X  =new_data.drop("Transported",axis=1)

y = new_data["Transported"]

In [140]:
num=X.select_dtypes(include="number").columns
cat=X.select_dtypes(include="object").columns

In [141]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [142]:
num_pipeline = Pipeline(steps=[("num_imputer", SimpleImputer(strategy="mean",missing_values=np.nan)),
                               ("scaler",StandardScaler())
                        ])

cat_pipeline = Pipeline(steps=[
                               ('imputer',SimpleImputer(strategy="most_frequent",missing_values=np.nan)),
                               ('onehot',OneHotEncoder(handle_unknown="ignore",sparse=False))
])

In [143]:
preprocessor = ColumnTransformer([("numerical_pipeline",num_pipeline,num),
                                  ("categorical_pipeline",cat_pipeline,cat)
                                  ])

In [144]:
pipe = Pipeline(steps=[
                       ("preprocessor",preprocessor),
                       
                       ("clf",LogisticRegression(max_iter=300))
                       
                       

])

In [145]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numerical_pipeline',
                                                  Pipeline(steps=[('num_imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['CryoSleep', 'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall',
       'Spa', 'VRDeck', 'Group', 'Group_size', 'IsAlone', 'Total_billed',
       'Age_group', 'cabin_param_1', 'cabin_param_3'],
      dtype='object')),
                                                 ('categorical_pipeline',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                     

In [146]:
y_pred = pipe.predict(X_test)

In [147]:
from sklearn.metrics import accuracy_score

In [148]:
acc1 = accuracy_score(y_test,y_pred)

In [149]:
acc1

0.7780333525014376

In [150]:
import lightgbm as lgb

In [151]:
pipe2 = Pipeline(steps=[
                       ("preprocessor",preprocessor),
                       
                       ("lgb",lgb.LGBMClassifier())
                       
                       

])

In [152]:
pipe2.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'preprocessor', 'lgb', 'preprocessor__n_jobs', 'preprocessor__remainder', 'preprocessor__sparse_threshold', 'preprocessor__transformer_weights', 'preprocessor__transformers', 'preprocessor__verbose', 'preprocessor__verbose_feature_names_out', 'preprocessor__numerical_pipeline', 'preprocessor__categorical_pipeline', 'preprocessor__numerical_pipeline__memory', 'preprocessor__numerical_pipeline__steps', 'preprocessor__numerical_pipeline__verbose', 'preprocessor__numerical_pipeline__num_imputer', 'preprocessor__numerical_pipeline__scaler', 'preprocessor__numerical_pipeline__num_imputer__add_indicator', 'preprocessor__numerical_pipeline__num_imputer__copy', 'preprocessor__numerical_pipeline__num_imputer__fill_value', 'preprocessor__numerical_pipeline__num_imputer__missing_values', 'preprocessor__numerical_pipeline__num_imputer__strategy', 'preprocessor__numerical_pipeline__num_imputer__verbose', 'preprocessor__numerical_pipeline__scaler__copy', 'prep

In [153]:
params_grid = [
               {"lgb__num_leaves":[10,20,30,40,50,60,70,80,90,100]
               }

]

In [154]:
grid_search = GridSearchCV(pipe2,param_grid=params_grid,scoring="accuracy",verbose=1)

In [155]:
grid_search.fit(X_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


GridSearchCV(estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('numerical_pipeline',
                                                                         Pipeline(steps=[('num_imputer',
                                                                                          SimpleImputer()),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         Index(['CryoSleep', 'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall',
       'Spa', 'VRDeck', 'Group', 'Group_size', 'IsAlone', 'Total_billed',
       'Age_group', 'cabin_param_1', 'c...
                                                                        ('categorical_pipeline',
                                                            

In [156]:
grid_search.best_params_

{'lgb__num_leaves': 30}

In [157]:
y_pred2 = grid_search.predict(X_test)

In [158]:
acc2 = accuracy_score(y_test,y_pred2)

In [159]:
acc2

0.8033352501437608